# Visdom Example

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

In [3]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


## Text

In [4]:
vis.text("Hello, world!", env="main")

'window_396015d424e87a'

## Image

In [5]:
a = torch.randn(3, 200, 200)
vis.image(a)

'window_396015e9e1d094'

## Images

In [6]:
vis.images(torch.Tensor(3, 3, 28, 28))

'window_396016049f0070'

## Example (using MNIST and CIFAR10)

In [7]:
MNIST = dsets.MNIST(root='./MNIST_data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root='./cifar10', train=True, transform=torchvision.transforms.ToTensor(), download=True)

100.0%

Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10


### CIFAR10

In [8]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([3, 32, 32])


'window_3960173c37e132'

### MNIST

In [9]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_3960173c3c7c34'

#### Check dataset

In [10]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                         batch_size=32,
                                         shuffle=False)

In [12]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


## Line Plot

In [13]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data) # y date만 찍을 경우 x축은 항상 0부터 1

In [14]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

In [15]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_396017785a6bc4'

### multiple Line on single windows

In [16]:
num = torch.Tensor(list(range(0, 10)))
print(num.shape)
num = num.view(-1, 1)
print(num.shape)
num = torch.cat((num, num), dim=1)
print(num.shape)

plt = vis.line(Y=torch.randn(10,2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


### Line info

In [17]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title='Test', showlegend=True))

In [18]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title='Test', legend=['1번'], showlegend=True))

In [19]:
plt = vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번','2번'], showlegend=True))

## Make function for update line

In [20]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value are Tensor'''
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update='append' 
            )

In [21]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## Close the window

In [22]:
vis.close(env="main")

''

# MNIST-CNN with Visdom

In [23]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

## Import visdom

In [24]:
import visdom

vis = visdom.Visdom()
vis.close(env='main')

Setting up a new session...


''

## define loss_tracker

In [25]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value are Tensor'''
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update='append' 
            )

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

device: cpu


In [27]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [28]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [29]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last=True)

In [32]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [33]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


In [34]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## Make plot

In [35]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

## Train with loss_tracker

In [36]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

[Epoch:1] cost = 0.118808314204216
[Epoch:2] cost = 0.04124435409903526
[Epoch:3] cost = 0.029922250658273697
[Epoch:4] cost = 0.021540004760026932
[Epoch:5] cost = 0.01778082735836506
[Epoch:6] cost = 0.01541284192353487
[Epoch:7] cost = 0.011932549998164177
[Epoch:8] cost = 0.010996047407388687
[Epoch:9] cost = 0.01011622417718172
[Epoch:10] cost = 0.00893942080438137
[Epoch:11] cost = 0.008273620158433914
[Epoch:12] cost = 0.006522528361529112
[Epoch:13] cost = 0.006921457126736641
[Epoch:14] cost = 0.00961926206946373
[Epoch:15] cost = 0.0055571505799889565
Learning Finished!


In [37]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

C:\Users\leeej\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\leeej\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9710000157356262
